# TSPLIB permutation benchmark
Runs NSGA-II across KroA?E100 instances comparing available backends.

In [ ]:
import sys
from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import types

def resolve_project_root(markers=("pyproject.toml", ".git")):
    current = Path.cwd().resolve()
    for candidate in (current, *current.parents):
        if any((candidate / marker).exists() for marker in markers):
            return candidate
    raise RuntimeError("No se pudo ubicar la raiz del proyecto; ejecuta este notebook dentro del arbol de VAMOS.")

PROJECT_ROOT = resolve_project_root()
SRC_DIR = PROJECT_ROOT / 'src'
if SRC_DIR.exists() and str(SRC_DIR) not in sys.path:
    sys.path.insert(0, str(SRC_DIR))

import vamos.runner as runner
from vamos.runner import ExperimentConfig, resolve_kernel
from vamos.external import run_external
from vamos.problem.registry import make_problem_selection
from vamos.study.runner import StudyRunner, StudyTask

plt.style.use('seaborn-v0_8-darkgrid')


In [ ]:
DEFAULT_CONFIG = ExperimentConfig()
TSPLIB_PROBLEMS = ['kroa100', 'krob100', 'kroc100', 'krod100', 'kroe100']
ACTIVE_PROBLEMS = list(TSPLIB_PROBLEMS)
PROBLEM_GRID = []

SEEDS = [DEFAULT_CONFIG.seed]
ALG = 'nsgaii'
INTERNAL_ENGINES = ['numpy', 'numba', 'moocore']
EXTERNAL_ALGOS = ['pymoo_perm_nsga2', 'jmetalpy_perm_nsga2']
INCLUDE_EXTERNAL = True
ENABLED_ENGINES = []


def detect_internal_engines(candidates):
    engines = []
    for name in candidates:
        try:
            resolve_kernel(name)
        except Exception as exc:
            print(f"Skipping engine '{name}': {exc}")
        else:
            engines.append(name)
    return engines


def set_active_problems(names):
    normalized = []
    for name in names:
        key = name.lower()
        if key not in TSPLIB_PROBLEMS:
            raise ValueError(f"Unknown TSPLIB benchmark '{name}'.")
        normalized.append(key)
    if not normalized:
        raise ValueError('At least one benchmark is required.')
    specs = [
        {
            'problem': key,
            'label': f"TSPLIB {key.upper()}",
        }
        for key in normalized
    ]
    txt = ', '.join(spec['label'] for spec in specs)
    print(f"Active problems: {txt}")
    return specs


ENABLED_ENGINES = detect_internal_engines(INTERNAL_ENGINES)
if not ENABLED_ENGINES:
    raise RuntimeError('No compatible engines detected.')
PROBLEM_GRID = set_active_problems(ACTIVE_PROBLEMS)
print('Engines:', ', '.join(ENABLED_ENGINES))
print('External algos:', ', '.join(EXTERNAL_ALGOS) if INCLUDE_EXTERNAL else 'None')


In [ ]:

RUN_HISTORY = []


def build_tasks(problem_grid, engines, seeds, include_external=True):
    tasks = []
    for spec in problem_grid:
        for engine in engines:
            for seed in seeds:
                tasks.append(StudyTask(
                    algorithm=ALG,
                    engine=engine,
                    problem=spec['problem'],
                    n_var=spec.get('n_var'),
                    seed=seed,
                ))
        if include_external:
            for algo in EXTERNAL_ALGOS:
                tasks.append(StudyTask(
                    algorithm=algo,
                    engine='external',
                    problem=spec['problem'],
                    n_var=spec.get('n_var'),
                    seed=seeds[0],
                ))
    return tasks


def run_tasks(tasks):
    global RUN_HISTORY
    runner = StudyRunner(verbose=True)
    raw_results = runner.run(tasks)
    RUN_HISTORY = [types.SimpleNamespace(selection=res.selection, metrics=res.metrics) for res in raw_results]
    rows = []
    for entry in RUN_HISTORY:
        m = entry.metrics
        rows.append({
            'problem': entry.selection.spec.key,
            'problem_label': entry.selection.spec.label,
            'engine': m['engine'],
            'algorithm': m['algorithm'],
            'time_ms': m['time_ms'],
            'evaluations': m['evaluations'],
            'evals_per_sec': m['evals_per_sec'],
            'hv': m.get('hv'),
        })
    return pd.DataFrame(rows)


In [ ]:
tasks = build_tasks(PROBLEM_GRID, ENABLED_ENGINES, SEEDS, include_external=False)
summary_df = run_tasks(tasks)
if INCLUDE_EXTERNAL:
    extra_rows = []
    ext_cfg = ExperimentConfig(seed=SEEDS[0], population_size=DEFAULT_CONFIG.population_size, max_evaluations=DEFAULT_CONFIG.max_evaluations)
    for spec in PROBLEM_GRID:
        selection = make_problem_selection(spec['problem'], n_var=spec.get('n_var'))
        for algo in EXTERNAL_ALGOS:
            metrics = run_external(
                algo,
                selection,
                use_native_problem=False,
                config=ext_cfg,
                make_metrics=runner._make_metrics,
                print_banner=lambda problem, selection, label, backend, cfg=ext_cfg: runner._print_run_banner(
                    problem, selection, label, backend, cfg
                ),
                print_results=runner._print_run_results,
            )
            if metrics is None:
                continue
            entry = types.SimpleNamespace(selection=selection, metrics=metrics)
            RUN_HISTORY.append(entry)
            extra_rows.append({
                'problem': selection.spec.key,
                'problem_label': selection.spec.label,
                'engine': metrics.get('engine'),
                'algorithm': metrics.get('algorithm'),
                'time_ms': metrics.get('time_ms'),
                'evaluations': metrics.get('evaluations'),
                'evals_per_sec': metrics.get('evals_per_sec'),
                'hv': metrics.get('hv'),
            })
    if extra_rows:
        extra_df = pd.DataFrame(extra_rows)
        summary_df = pd.concat([summary_df, extra_df], ignore_index=True)
summary_df


In [ ]:

if summary_df.empty:
    print('No data yet.')
else:
    for label, group in summary_df.groupby('problem_label', sort=False):
        fig, axes = plt.subplots(1, 2, figsize=(12, 4))
        axes[0].bar(group['algorithm'], group['hv'], color='tab:blue')
        axes[0].set_ylabel('Hypervolume')
        axes[0].set_title(label)
        axes[0].tick_params(axis='x', rotation=30)

        axes[1].bar(group['algorithm'], group['time_ms'], color='tab:orange')
        axes[1].set_ylabel('Time (ms)')
        axes[1].set_title(label)
        axes[1].tick_params(axis='x', rotation=30)

        fig.suptitle(label)
        fig.tight_layout()
        plt.show()


In [ ]:

if not RUN_HISTORY:
    print('Run the experiment cell first.')
else:
    runs_by_problem = {}
    for res in RUN_HISTORY:
        label = res.selection.spec.label
        runs_by_problem.setdefault(label, []).append(res)
    for label, runs in runs_by_problem.items():
        fig, ax = plt.subplots(figsize=(6, 5))
        for res in runs:
            F = res.metrics['F']
            legend_entry = f"{res.metrics['algorithm']}"
            ax.scatter(F[:, 0], F[:, 1], s=30, alpha=0.7, label=legend_entry)
        ax.set_xlabel('Tour length')
        ax.set_ylabel('Max edge length')
        ax.set_title(f'{label} Pareto front')
        ax.legend()
        ax.grid(True, alpha=0.3)
        plt.tight_layout()
        plt.show()
